In [ ]:

"""
IA analysis 
100 in timber and 300 acre in grasslands 

CITATION FOR THIS LOGIC:
*!*!
https://www.nifc.gov/sites/default/files/redbook-files/Chapter11.pdf
*!*!

Fire-size class definitions (by final fire perimeter acres)
-----------------------------------------------------------
A: > 0 and <= 0.25 acres
B: >= 0.26 and <= 9.9 acres
C: >= 10.0 and <= 99.9 acres
D: >= 100 and <= 299 acres
E: >= 300 and <= 999 acres
F: >= 1000 and <= 4999 acres
G: >= 5000 acres

Rules for `event_type`:
- If fire occurs in land cover types [52 (Shrubland), 71 (Grassland), 81 (Pasture/Hay)] (300 acres in grasslands ):
    - Assign event_type = 1 only if FIRE_SIZE_CLASS is in ['E', 'F', 'G'] (larger fires).

- If fire occurs in any other land cover (100 acres in forests):
    - Assign event_type = 1 if FIRE_SIZE_CLASS is in ['D', 'E', 'F', 'G'].

- Otherwise assign event_type = 0.
"""

import pandas as pd
import matplotlib.pyplot as plt

# # Read CSV
# df = pd.read_csv(r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\2020_FPA_FOD_cons.csv")

# # Display head
# # print(df.head())

# # Create binary event_type column based on FIRE_SIZE_CLASS
# #df['event_type'] = df['FIRE_SIZE_CLASS'].apply(lambda x: 1 if x in ['D', 'E', 'F', 'G'] else 0)
# df['event_type'] = df.apply(
#     lambda row: 1 if (row['Land_Cover'] in [52, 71, 81] and row['FIRE_SIZE_CLASS'] in ['E', 'F', 'G']) 
#     or (row['Land_Cover'] not in [52, 71, 81] and row['FIRE_SIZE_CLASS'] in ['D', 'E', 'F', 'G']) 
#     else 0,
#     axis=1
# )

# Display head
import pandas as pd
import numpy as np

csv_path = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\FPA_FOD_Plus.csv"

# How many rows to sample (e.g., 1% of the file)
frac =  1#0.001  

# Count lines quickly
n_lines = sum(1 for _ in open(csv_path))

# Build random skip mask (keep header row)
skip = sorted(np.random.choice(np.arange(1, n_lines), size=int((n_lines-1)*(1-frac)), replace=False))

df_FULL = pd.read_csv(csv_path, skiprows=skip)

print(df_FULL.shape)

# Create binary event_type column based on FIRE_SIZE_CLASS
#df['event_type'] = df['FIRE_SIZE_CLASS'].apply(lambda x: 1 if x in ['D', 'E', 'F', 'G'] else 0)
df_FULL['event_type'] = df_FULL.apply(
    lambda row: 1 if (row['Land_Cover'] in [52, 71, 81] and row['FIRE_SIZE_CLASS'] in ['E', 'F', 'G']) 
    or (row['Land_Cover'] not in [52, 71, 81] and row['FIRE_SIZE_CLASS'] in ['D', 'E', 'F', 'G']) 
    else 0,
    axis=1
)

print(len(df_FULL))

df_SUB = df_FULL[df_FULL["NWCG_REPORTING_AGENCY"].isin(["BLM", "NPS", "FS"])]
print(df_SUB["NWCG_REPORTING_AGENCY"].value_counts())
print(len(df_SUB))

In [ ]:
# df_SUB = df_FULL[df_FULL["NWCG_REPORTING_AGENCY"].isin(["BLM", "NPS", "FS"])]
# print(df_SUB["NWCG_REPORTING_AGENCY"].value_counts())
# print(len(df_SUB))

In [ ]:
# Unique counts for GACCAbbrev
print("Value counts (including NaN):")
print(df_SUB["GACCAbbrev"].value_counts(dropna=False))

print("\nUnique values:")
print(df_SUB["GACCAbbrev"].unique())

print("\nNumber of unique values (including NaN):")
print(df_SUB["GACCAbbrev"].nunique(dropna=False))

# Check for "wrong"/unexpected values
valid_set = {"ONCC","GBCC","NWCC","SWCC","NRCC","AICC","SACC","RMCC","OSCC","EACC"}
mask_invalid = ~df_SUB["GACCAbbrev"].isin(valid_set) & df_SUB["GACCAbbrev"].notna()

if mask_invalid.any():
    print("\n⚠️ Unexpected GACCAbbrev values found:")
    print(df_SUB.loc[mask_invalid, "GACCAbbrev"].value_counts())
else:
    print("\nAll non-null GACCAbbrev values are in the expected set.")


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

# --- Start from merged_final and ensure no sjoin-reserved columns are present ---
df = df_SUB.copy()
for col in ["index_right", "index_left"]:
    if col in df.columns:
        df = df.drop(columns=col)

# --- Keep only rows with valid coords ---
mask_xy = df["LATITUDE"].notna() & df["LONGITUDE"].notna()
df_xy = df.loc[mask_xy].copy()

# --- Build point GeoDataFrame ---
gdf_points = gpd.GeoDataFrame(
    df_xy,
    geometry=gpd.points_from_xy(df_xy["LONGITUDE"], df_xy["LATITUDE"]),
    crs="EPSG:4326"
)

# --- Load GACC polygons ---
gacc_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\National_GACC_Final_20250113.shp"
gdf_gacc = gpd.read_file(gacc_fp)[["GACCAbbrev", "geometry"]].reset_index(drop=True)

# --- Reproject if needed ---
if gdf_gacc.crs != gdf_points.crs:
    gdf_gacc = gdf_gacc.to_crs(gdf_points.crs)

# --- Spatial join (avoid 'index_right' collision via suffixes) ---
gdf_joined = gpd.sjoin(
    gdf_points.reset_index(),               # keep original index to map back
    gdf_gacc,
    how="left",
    predicate="within",
    lsuffix="PTS",
    rsuffix="GACC"
)

# --- Extract GACCAbbrev as GACCTRUTH and map back to full dataframe ---
gacc_map = gdf_joined.set_index("index")["GACCAbbrev"]
df["GACCTRUTH"] = np.nan
df.loc[gacc_map.index, "GACCTRUTH"] = gacc_map.values

# --- Put result back into merged_final ---
df_SUB = df

# Optional quick check:
# print(merged_final["GACCTRUTH"].value_counts(dropna=False))


In [ ]:
"""
This script summarizes initial-attack (IA) outcomes from a DataFrame that already has event_type coded (0 = IA success, 1 = IA failure). 
It first prints overall counts and percentages of successes and failures, then builds a bar chart showing total counts with 
percentage labels for each outcome. If the GACCAbbrev column exists, it groups by GACC, computes each region’s success/failure counts and shares, 
sorts regions by success share, and plots a stacked bar chart (success on bottom, failure on top) with per-region success percentages labeled above the bars. 
If GACCAbbrev is missing, it simply skips the by-GACC plots.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assumes df already exists with column 'event_type' (0=IA success, 1=IA failure)
# If not, uncomment and set your CSV path:
# df = pd.read_csv(r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\2020_FPA_FOD_cons.csv")

df= df_SUB

# ---------- 1) Print success/failure ratio ----------
total = len(df)
n_success = int((df["event_type"] == 0).sum())
n_failure = int((df["event_type"] == 1).sum())
print(f"IA Success (0) / Total: {n_success} / {total}  ({(n_success/total*100):.2f}%)")
print(f"IA Failure (1) / Total: {n_failure} / {total}  ({(n_failure/total*100):.2f}%)")

# ---------- 2) Overall counts + percentages ----------
counts = df["event_type"].value_counts().reindex([0,1]).fillna(0).astype(int)
props = counts / counts.sum()

fig, ax = plt.subplots(figsize=(7,4.5))
bars = ax.bar(["IA Success", "IA Failure"], counts.values, color=["#2ca02c", "#d62728"])
for rect, c, p in zip(bars, counts.values, props.values):
    ax.text(rect.get_x() + rect.get_width()/2, rect.get_height() * 1.01,
            f"{c:,}\n({p*100:.1f}%)", ha="center", va="bottom", fontsize=12)
#ax.set_title("Initial Attack Outcomes (Counts with %)")
#ax.set_ylabel("Number of Fires")
# After creating your plot and before plt.tight_layout(), format y-axis in thousands:
import matplotlib.ticker as mtick

ax.set_ylabel("Number of Fires (thousands)")
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _ : f"{x/1000:.1f}k"))

ax.set_ylim(0, counts.max() * 1.25 if counts.max() > 0 else 1)
ax.grid(axis="y", alpha=0.2, linestyle="--")
plt.tight_layout()
plt.savefig(r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\FIGURES\IA_SUCCESS_FIRESIZE_FULL_FED.png", dpi=300, bbox_inches="tight")
plt.show()

# ---------- 3) By GACCAbbrev (stacked shares and success-rate bars) ----------
gacc_col = "GACCAbbrev"
if gacc_col in df.columns:
    dfg = df.dropna(subset=[gacc_col]).copy()

    gb = dfg.groupby(gacc_col)["event_type"]
    by_counts = gb.value_counts().unstack(fill_value=0).reindex(columns=[0,1], fill_value=0)
    by_counts["total"] = by_counts.sum(axis=1)

    # Compute shares
    by_share = by_counts[[0,1]].div(by_counts["total"].replace(0, np.nan), axis=0).fillna(0)

    # Sort by IA success share descending
    by_share_sorted = by_share.sort_values(0, ascending=False)
    by_counts_sorted = by_counts.loc[by_share_sorted.index]

    # --- 3a) Stacked share (IA success vs failure) by GACC ---
    fig, ax = plt.subplots(figsize=(10, max(4.5, 0.5*len(by_share_sorted))))
    ax.bar(by_share_sorted.index, by_share_sorted[0].values, label="IA Success", color="#2ca02c")
    ax.bar(by_share_sorted.index, by_share_sorted[1].values, bottom=by_share_sorted[0].values, label="IA Failure", color="#d62728")
    for i, g in enumerate(by_share_sorted.index):
        succ_pct = by_share_sorted.loc[g, 0] * 100
        ax.text(i, 1.005, f"{succ_pct:.0f}%", ha="center", va="bottom", fontsize=9)
    #ax.set_title("IA Outcomes by GACC")
    ax.set_ylabel("Proportion")
    ax.set_ylim(0, 1.15)
    ax.set_xticks(range(len(by_share_sorted.index)))
    ax.set_xticklabels(by_share_sorted.index, rotation=45, ha="right")
    ax.legend(loc="upper left", bbox_to_anchor=(1.01, 1.02))
    ax.grid(axis="y", alpha=0.2, linestyle="--")
    plt.tight_layout()
    plt.savefig(r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\FIGURES\IA_SUCCESS_FIRESIZE_GACC_FED.png", dpi=300, bbox_inches="tight")
    plt.show()


else:
    print("Column 'GACCAbbrev' not found; skipping by-GACC plots.")


In [ ]:
### NEED 24 hour


In [ ]:
# ---------- Aggregate ----------
yearly = (
    df.groupby("FIRE_YEAR")["event_type"]
      .agg(total="count", large="sum")
      .reset_index()
      .sort_values("FIRE_YEAR")
)
yearly = yearly[(yearly["FIRE_YEAR"] >= 1992) & (yearly["FIRE_YEAR"] <= 2020)]
yearly["small"] = yearly["total"] - yearly["large"]

# ---------- Plot: line chart ----------
fig, ax = plt.subplots(figsize=(12,6))
x = yearly["FIRE_YEAR"].values

# Lines with new colors
ax.plot(x, yearly["small"].values, marker="o", color="#2ca02c", label="Small Fires")
ax.plot(x, yearly["large"].values, marker="o", color="#d62728", label="Large Fires")

# Labels and formatting
ax.set_ylabel("Number of Ignitions", fontsize=12)
ax.set_xlabel("Year", fontsize=12)
ax.tick_params(axis="both", which="major", labelsize=12)
ax.grid(True, linestyle="--", alpha=0.4, axis="y")

# Legend outside to the right
ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), fontsize=12, frameon=False)

ax.set_ylim(0, max(yearly["small"].max(), yearly["large"].max()) * 1.05)

# --- Custom x-axis ticks ---
ax.set_xticks([1992, 1995, 2000, 2005, 2010, 2015, 2020])

plt.tight_layout()
plt.savefig(r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\FIGURES\IA_TrendsOverTime_92_2020.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
# ERC
import pandas as pd
import matplotlib.pyplot as plt

# Drop NA in ERC
df_bin = df.dropna(subset=["erc"]).copy()

# Define bins (percentile thresholds)
bins = [0, 80, 90, 95, 97.5, 100]
labels = ["0–80", "80–90", "90–95", "95–97.5", "97.5–100"]

# Compute percentile cutoffs from the data
cutoffs = np.percentile(df_bin["erc"], bins)

# Bin ERC values
df_bin["erc_bin"] = pd.cut(df_bin["erc"], bins=cutoffs, labels=labels, include_lowest=True)

# Count successes (0) vs failures (1) in each bin
counts = df_bin.groupby(["erc_bin", "event_type"]).size().unstack(fill_value=0)

# Side-by-side bar plot
fig, ax = plt.subplots(figsize=(10,6))
counts.plot(kind="bar", ax=ax, width=0.8, color={0:"#2ca02c", 1:"#d62728"})

ax.set_xlabel("ERC Percentile Bin", fontsize=12)
ax.set_ylabel("Number of Fires", fontsize=12)
ax.set_title("IA Success vs Failure by ERC Percentile Bin", fontsize=14)
ax.legend(["Success", "Failure"], fontsize=12)
ax.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Drop NA in VPD
df_bin = df.dropna(subset=["vpd"]).copy()

# Define bins (percentile thresholds)
bins = [0, 80, 90, 95, 97.5, 100]
labels = ["0–80", "80–90", "90–95", "95–97.5", "97.5–100"]

# Compute percentile cutoffs from the data
cutoffs = np.percentile(df_bin["vpd"], bins)

# Bin VPD values
df_bin["vpd_bin"] = pd.cut(df_bin["vpd"], bins=cutoffs, labels=labels, include_lowest=True)

# Count successes (0) vs failures (1) in each bin
counts = df_bin.groupby(["vpd_bin", "event_type"]).size().unstack(fill_value=0)

# Side-by-side bar plot
fig, ax = plt.subplots(figsize=(10,6))
counts.plot(kind="bar", ax=ax, width=0.8, color={0:"#2ca02c", 1:"#d62728"})

ax.set_xlabel("VPD Percentile Bin", fontsize=12)
ax.set_ylabel("Number of Fires", fontsize=12)
ax.set_title("IA Success vs Failure by VPD Percentile Bin", fontsize=14)
ax.legend(["Success", "Failure"], fontsize=12)
ax.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
# Replace extreme negative SDI fill values with 0
df.loc[df["SDI"] < -1e20, "SDI"] = 0

# Check results
print("SDI min:", df["SDI"].min())
print("SDI max:", df["SDI"].max())


In [ ]:
df = df_SUB

In [ ]:
# === IA success as a function of VPD and ERC: scatter/hexbin + failure-rate heatmap ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# -------------------------------------------------------------------
# Prep: keep rows with both VPD and ERC and event_type in {0,1}
# -------------------------------------------------------------------
d = df.loc[df["vpd"].notna() & df["erc"].notna() & df["event_type"].isin([0,1]), ["vpd","erc","event_type"]].copy()

# -------------------------------------------------------------------
# 1) Hexbin “scatter-like” density, colored by count (two panels)
#    Left = IA Success (event_type=0), Right = IA Failure (event_type=1)
# -------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)
for ax, et, title in zip(
    axes,
    [0, 1],
    ["IA Success (event_type=0)", "IA Failure (event_type=1)"]
):
    dd = d[d["event_type"] == et]
    hb = ax.hexbin(
        dd["vpd"].values, dd["erc"].values,
        gridsize=50, mincnt=1, linewidths=0.0, cmap="viridis"
    )
    ax.set_xlabel("VPD", fontsize=12)
    ax.set_ylabel("ERC", fontsize=12)
    ax.tick_params(labelsize=12)
    ax.set_title(title, fontsize=12)
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label("Count", fontsize=12)
plt.tight_layout()
# plt.savefig(r"C:\path\to\FIGURES\vpd_erc_hexbin_success_failure.png", dpi=300, bbox_inches="tight")
plt.show()

# -------------------------------------------------------------------
# 2) 2D binned FAILURE RATE heatmap over VPD×ERC
#    Color shows share of failures (event_type=1) in each (VPD, ERC) bin
# -------------------------------------------------------------------
# Define bin edges (adjust ranges as needed)
vpd_bins = np.linspace(d["vpd"].quantile(0.01), d["vpd"].quantile(0.99), 30)
erc_bins = np.linspace(d["erc"].quantile(0.01), d["erc"].quantile(0.99), 30)

# Assign bins
vpd_bin_idx = np.digitize(d["vpd"].values, vpd_bins) - 1
erc_bin_idx = np.digitize(d["erc"].values, erc_bins) - 1

# Keep only indices that fall into valid bin ranges
valid_mask = (vpd_bin_idx >= 0) & (vpd_bin_idx < len(vpd_bins)-1) & (erc_bin_idx >= 0) & (erc_bin_idx < len(erc_bins)-1)
vb = vpd_bin_idx[valid_mask]
eb = erc_bin_idx[valid_mask]
evt = d["event_type"].values[valid_mask]

# Count totals and failures per cell
tot_grid = np.zeros((len(erc_bins)-1, len(vpd_bins)-1), dtype=np.int32)
fail_grid = np.zeros_like(tot_grid)
np.add.at(tot_grid, (eb, vb), 1)
np.add.at(fail_grid, (eb, vb), (evt == 1).astype(int))

# Failure rate (avoid divide-by-zero)
with np.errstate(divide="ignore", invalid="ignore"):
    fail_rate = np.where(tot_grid > 0, fail_grid / tot_grid, np.nan)

# Plot heatmap
fig, ax = plt.subplots(figsize=(7.5, 6))
im = ax.imshow(
    fail_rate,
    origin="lower",
    aspect="auto",
    extent=[vpd_bins[0], vpd_bins[-1], erc_bins[0], erc_bins[-1]],
    cmap="Reds",
    vmin=0.0, vmax=1.0,
)
ax.set_xlabel("VPD", fontsize=12)
ax.set_ylabel("ERC", fontsize=12)
ax.tick_params(labelsize=12)
ax.set_title("IA Failure Rate by VPD × ERC (share of failures)", fontsize=12)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Failure rate (0–1)", fontsize=12)

# Optional: overlay sparse contours of count to show data density
# import matplotlib
# cs = ax.contour(
#     (vpd_bins[:-1][None, :]+vpd_bins[1:][None, :])/2,
#     (erc_bins[:-1][:, None]+erc_bins[1:][:, None])/2,
#     tot_grid, levels=5, linewidths=0.6, colors="k", alpha=0.4
# )

plt.tight_layout()
# plt.savefig(r"C:\path\to\FIGURES\vpd_erc_failure_rate_heatmap.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:

from matplotlib.colors import PowerNorm

# -------------------------------------------------------------------
# 2) 2D binned FAILURE RATE heatmap over VPD×ERC
#    Color shows share of failures (event_type=1) in each (VPD, ERC) bin
# -------------------------------------------------------------------
# Define bin edges (adjust ranges as needed)
vpd_bins = np.linspace(d["vpd"].quantile(0.01), d["vpd"].quantile(0.99), 75)
erc_bins = np.linspace(d["erc"].quantile(0.01), d["erc"].quantile(0.99), 75)

# Assign bins
vpd_bin_idx = np.digitize(d["vpd"].values, vpd_bins) - 1
erc_bin_idx = np.digitize(d["erc"].values, erc_bins) - 1

# Keep only indices that fall into valid bin ranges
valid_mask = (vpd_bin_idx >= 0) & (vpd_bin_idx < len(vpd_bins)-1) & (erc_bin_idx >= 0) & (erc_bin_idx < len(erc_bins)-1)
vb = vpd_bin_idx[valid_mask]
eb = erc_bin_idx[valid_mask]
evt = d["event_type"].values[valid_mask]

# Count totals and failures per cell
tot_grid = np.zeros((len(erc_bins)-1, len(vpd_bins)-1), dtype=np.int32)
fail_grid = np.zeros_like(tot_grid)
np.add.at(tot_grid, (eb, vb), 1)
np.add.at(fail_grid, (eb, vb), (evt == 1).astype(int))

# Failure rate (avoid divide-by-zero)
with np.errstate(divide="ignore", invalid="ignore"):
    fail_rate = np.where(tot_grid > 0, fail_grid / tot_grid, np.nan)

# Plot heatmap
fig, ax = plt.subplots(figsize=(7.5, 6))
im = ax.imshow(
    fail_rate,
    origin="lower",
    aspect="auto",
    extent=[vpd_bins[0], vpd_bins[-1], erc_bins[0], erc_bins[-1]],
    cmap="Reds",
    vmin=0.0, vmax=1.0,
)
ax.set_xlabel("VPD", fontsize=12)
ax.set_ylabel("ERC", fontsize=12)
ax.tick_params(labelsize=12)
ax.set_title("IA Failure Rate by VPD × ERC (share of failures)", fontsize=12)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Failure rate (0–1)", fontsize=12)

# Optional: overlay sparse contours of count to show data density
# import matplotlib
# cs = ax.contour(
#     (vpd_bins[:-1][None, :]+vpd_bins[1:][None, :])/2,
#     (erc_bins[:-1][:, None]+erc_bins[1:][:, None])/2,
#     tot_grid, levels=5, linewidths=0.6, colors="k", alpha=0.4
# )

plt.tight_layout()
# plt.savefig(r"C:\path\to\FIGURES\vpd_erc_failure_rate_heatmap.png", dpi=300, bbox_inches="tight")
plt.show()




# === SDI vs VPD failure-rate heatmap ===
d_sdi = df.loc[df["SDI"].notna() & df["vpd"].notna() & df["event_type"].isin([0,1]),
               ["SDI","vpd","event_type"]].copy()

# Define bin edges
sdi_bins = np.linspace(d_sdi["SDI"].quantile(0.01), d_sdi["SDI"].quantile(0.99), 75)
vpd_bins = np.linspace(d_sdi["vpd"].quantile(0.01), d_sdi["vpd"].quantile(0.99), 75)

# Assign bins
sdi_idx = np.digitize(d_sdi["SDI"].values, sdi_bins) - 1
vpd_idx = np.digitize(d_sdi["vpd"].values, vpd_bins) - 1
valid = (sdi_idx >= 0) & (sdi_idx < len(sdi_bins)-1) & (vpd_idx >= 0) & (vpd_idx < len(vpd_bins)-1)

# Totals + failures
tot = np.zeros((len(sdi_bins)-1, len(vpd_bins)-1), dtype=int)
fail = np.zeros_like(tot)
np.add.at(tot, (sdi_idx[valid], vpd_idx[valid]), 1)
np.add.at(fail, (sdi_idx[valid], vpd_idx[valid]), (d_sdi["event_type"].values[valid] == 1).astype(int))

fail_rate = np.where(tot > 0, fail/tot, np.nan)

# Plot
fig, ax = plt.subplots(figsize=(7.5,6))
im = ax.imshow(fail_rate, origin="lower", aspect="auto",
               extent=[vpd_bins[0], vpd_bins[-1], sdi_bins[0], sdi_bins[-1]],
               cmap="Reds", vmin=0, vmax=1)

ax.set_xlabel("VPD", fontsize=12)
ax.set_ylabel("SDI", fontsize=12)
ax.set_title("IA Failure Rate by SDI × VPD", fontsize=12)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Failure rate", fontsize=12)
plt.tight_layout()
plt.show()


# === SDI vs ERC failure-rate heatmap ===
d_sdi = df.loc[df["SDI"].notna() & df["erc"].notna() & df["event_type"].isin([0,1]),
               ["SDI","erc","event_type"]].copy()

sdi_bins = np.linspace(d_sdi["SDI"].quantile(0.01), d_sdi["SDI"].quantile(0.99), 75)
erc_bins = np.linspace(d_sdi["erc"].quantile(0.01), d_sdi["erc"].quantile(0.99), 75)

sdi_idx = np.digitize(d_sdi["SDI"].values, sdi_bins) - 1
erc_idx = np.digitize(d_sdi["erc"].values, erc_bins) - 1
valid = (sdi_idx >= 0) & (sdi_idx < len(sdi_bins)-1) & (erc_idx >= 0) & (erc_idx < len(erc_bins)-1)

tot = np.zeros((len(sdi_bins)-1, len(erc_bins)-1), dtype=int)
fail = np.zeros_like(tot)
np.add.at(tot, (sdi_idx[valid], erc_idx[valid]), 1)
np.add.at(fail, (sdi_idx[valid], erc_idx[valid]), (d_sdi["event_type"].values[valid] == 1).astype(int))

fail_rate = np.where(tot > 0, fail/tot, np.nan)

fig, ax = plt.subplots(figsize=(7.5,6))
im = ax.imshow(fail_rate, origin="lower", aspect="auto",
               extent=[erc_bins[0], erc_bins[-1], sdi_bins[0], sdi_bins[-1]],
               cmap="Reds", vmin=0, vmax=1)


ax.set_xlabel("ERC", fontsize=12)
ax.set_ylabel("SDI", fontsize=12)
ax.set_title("IA Failure Rate by SDI × ERC", fontsize=12)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Failure rate", fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Count how many SDI values are at the float32 fill value
bad_val = -3.4028234663852885e+36
n_bad = (df["SDI"] == bad_val).sum()

print("Number of SDI entries equal to fill value:", n_bad)
# Count how many SDI values are at the float32 fill value
bad_val = -3.4028234663852885e+36
n_bad = (df["SDI"] == bad_val).sum()

print("Number of SDI entries equal to fill value:", n_bad)


In [ ]:
# FAILURES
df_SUM_Failures = df_SUB[df_SUB["event_type"] == 1].copy()
df_SUM_SUCC = df_SUB[df_SUB["event_type"] == 0].copy()


In [ ]:
# df_SUM_Failures connects to For ia failures Connect to 209 =  ics209plus-wf_cpx-assoc-summary_2014to2023-draft ("C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_cpx-assoc-summary_2014to2023-draft.csv")
# Not counting complexes probablamitcally
# So need to column ‘y’ FOD_ID left join so that ‘e’ CPLX_INCIDENT_ID  

In [ ]:
import pandas as pd

# --- Load the ICS-209 summary file ---
ics209_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_cpx-assoc-summary_2014to2023-draft.csv"
ics209 = pd.read_csv(ics209_fp)

# --- Inspect available columns ---
print(ics209.columns)

# --- Join with df_SUM_Failures ---
# Assume df_SUM_Failures has column "FOD_ID"
# and ICS-209 file has "CPLX_INCIDENT_ID" that should link
merged = df_SUM_Failures.merge(
    ics209,
    how="left",
    left_on="FOD_ID",
    right_on="CPLX_INCIDENT_ID",
    suffixes=("", "_ics209")
)

# --- Drop complex records if needed (to avoid double-counting) ---
if "CPLX_FLG" in merged.columns:
    merged = merged.loc[merged["CPLX_FLG"] != 1]  # adjust flag logic as appropriate

print(merged.head())


In [ ]:
import pandas as pd

# Load ICS-209
ics209_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_cpx-assoc-summary_2014to2023-draft.csv"
ics209 = pd.read_csv(ics209_fp)

# Standardize merge keys to string
df_SUM_Failures["FOD_ID"] = df_SUM_Failures["FOD_ID"].astype(str)
ics209["FOD_ID"] = ics209["FOD_ID"].astype(str)

# Merge
merged = df_SUM_Failures.merge(
    ics209,
    how="left",
    left_on="FOD_ID",
    right_on="FOD_ID",
    suffixes=("", "_ics209")
)

print(merged.head())


In [ ]:
# import pandas as pd
# import numpy as np

# # --- Load ICS-209 files (as you already do) ---
# ics209_assoc_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_cpx-assoc-summary_2014to2023-draft.csv"
# ics209_incidents_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_incidents_1999to2023-draft.csv"

# ics209_assoc = pd.read_csv(ics209_assoc_fp)
# ics209_incidents = pd.read_csv(ics209_incidents_fp)

# # --- Make safe copies ---
# df_sf = df_SUM_Failures.copy()
# assoc = ics209_assoc.copy()
# inc   = ics209_incidents.copy()

# # --- Helper: normalize FOD_ID to pure digit strings (removes .0, spaces, non-digits) ---
# def normalize_fod(s: pd.Series) -> pd.Series:
#     out = s.astype(str).str.strip()
#     out = out.str.replace(r'(?i)^nan$', '', regex=True)     # turn 'nan' strings to empty
#     out = out.str.replace(r'\.0+$', '', regex=True)        # drop trailing .0 from floats
#     out = out.str.replace(r'\D', '', regex=True)           # keep only digits
#     out = out.where(out.ne(''), np.nan)                    # empty -> NaN
#     return out

# # --- Normalize FOD_ID on both sides ---
# df_sf["FOD_ID_norm"] = normalize_fod(df_sf["FOD_ID"])
# assoc["FOD_ID_norm"] = normalize_fod(assoc["FOD_ID"])

# # (Optional) keep only 2014+ since ICS-209 assoc is 2014–2023
# df_sf_2014p = df_sf.loc[df_sf["FIRE_YEAR"] >= 2014].copy()

# # --- Merge failures with assoc on normalized FOD_ID ---
# merged = df_sf_2014p.merge(
#     assoc.drop(columns=["FOD_ID"], errors="ignore"),
#     how="left",
#     left_on="FOD_ID_norm",
#     right_on="FOD_ID_norm",
#     indicator=True,
#     suffixes=("", "_assoc")
# )

# # --- Report match rate ---
# n_all = len(merged)
# n_matched = int((merged["_merge"] == "both").sum())
# print(f"Matched on FOD_ID (normalized): {n_matched:,} / {n_all:,} ({n_matched/n_all*100:.1f}%)")

# # --- Standardize & merge incident-level data (assoc 'CPLX_INCIDENT_ID' -> incidents 'INCIDENT_ID') ---
# if "CPLX_INCIDENT_ID" in assoc.columns and "INCIDENT_ID" in inc.columns:
#     merged["CPLX_INCIDENT_ID_norm"] = normalize_fod(merged.get("CPLX_INCIDENT_ID"))
#     inc["INCIDENT_ID_norm"] = normalize_fod(inc["INCIDENT_ID"])

#     merged = merged.merge(
#         inc.drop(columns=["INCIDENT_ID"], errors="ignore"),
#         how="left",
#         left_on="CPLX_INCIDENT_ID_norm",
#         right_on="INCIDENT_ID_norm",
#         suffixes=("", "_incident")
#     )

# # --- Peek ---
# print(merged[["FIRE_YEAR","FOD_ID","FOD_ID_norm","_merge"]].head(10))


In [ ]:
# import pandas as pd

# # --- Load ICS-209 summary (assoc) ---
# ics209_assoc_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_cpx-assoc-summary_2014to2023-draft.csv"
# ics209_assoc = pd.read_csv(ics209_assoc_fp)

# # --- Load ICS-209 incidents ---
# ics209_incidents_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_incidents_1999to2023-draft.csv"
# ics209_incidents = pd.read_csv(ics209_incidents_fp)

# # # --- Standardize join keys ---
# # df_SUM_Failures["FOD_ID"] = df_SUM_Failures["FOD_ID"].astype(str)
# # ics209_assoc["FOD_ID"] = ics209_assoc["FOD_ID"].astype(str)
# # ics209_assoc["CPLX_INC_IDENTIFIER"] = ics209_assoc["CPLX_INCIDENT_ID"].astype(str)
# # ics209_incidents["CPLX_INC_IDENTIFIER"] = ics209_incidents["INCIDENT_ID"].astype(str)

# # # --- Step 1: Failures + assoc ---
# # merged = df_SUM_Failures.merge(
# #     ics209_assoc,
# #     how="left",
# #     on="FOD_ID",
# #     suffixes=("", "_assoc")
# # )

# # # --- Step 2: Add incident-level info using CPLX_INCIDENT_ID ---
# # merged = merged.merge(
# #     ics209_incidents,
# #     how="left",
# #     on="CPLX_INC_IDENTIFIER",
# #     suffixes=("", "_incident")
# # )

# # print(merged.head())
# # Clean helpers
# def normalize_id(s: pd.Series) -> pd.Series:
#     s = s.astype("string")                         # keep real <NA>, not "nan"
#     s = s.str.strip()
#     s = s.str.replace(r'\.0+$', '', regex=True)    # remove trailing .0
#     s = s.str.replace(r'\D', '', regex=True)       # keep only digits
#     return s

# # Normalize FOD_ID on both sides
# df_SUM_Failures["FOD_ID_norm"] = normalize_id(df_SUM_Failures["FOD_ID"])
# ics209_assoc["FOD_ID_norm"]    = normalize_id(ics209_assoc["FOD_ID"])

# # Normalize complex/incident IDs, too
# ics209_assoc["CPLX_INCIDENT_ID_norm"] = normalize_id(ics209_assoc["CPLX_INCIDENT_ID"])
# ics209_incidents["INCIDENT_ID_norm"]  = normalize_id(ics209_incidents["INCIDENT_ID"])

# # --- Step 1: Failures + assoc on normalized FOD_ID ---
# merged = df_SUM_Failures.merge(
#     ics209_assoc,
#     how="left",
#     left_on="FOD_ID_norm",
#     right_on="FOD_ID_norm",
#     suffixes=("", "_assoc"),
#     indicator=True
# )

# # # --- Step 2: Add incident-level info using normalized IDs ---
# # merged = merged.merge(
# #     ics209_incidents,
# #     how="left",
# #     left_on="CPLX_INCIDENT_ID_norm",
# #     right_on="INCIDENT_ID_norm",
# #     suffixes=("", "_incident")
# # )

# # Make sure FIRE_SIZE is numeric
# merged["FIRE_SIZE_num"] = pd.to_numeric(merged["FIRE_SIZE"], errors="coerce")

# # --- First: merge on CPLX_INCIDENT_ID_norm ---
# m1 = merged.merge(
#     ics209_incidents,
#     how="left",
#     left_on="CPLX_INCIDENT_ID_norm",
#     right_on="INCIDENT_ID_norm",
#     suffixes=("", "_incident")
# )

# # --- Identify rows that did NOT merge on complex id ---
# no_match_mask = m1["INCIDENT_ID_norm"].isna()

# # --- Second: merge those rows on FOD_ID_norm instead ---
# m2 = merged.loc[no_match_mask].merge(
#     ics209_incidents,
#     how="left",
#     left_on="FOD_ID_norm",
#     right_on="FOD_ID_norm",   # need to normalize FOD_ID in ics209_incidents too
#     suffixes=("", "_incident")
# )

# # --- Combine back together ---
# merged_final = pd.concat([m1.loc[~no_match_mask], m2], ignore_index=True)


In [ ]:
len(merged)

In [ ]:
import pandas as pd

import pandas as pd

# --- Load ICS-209 summary (assoc) ---
ics209_assoc_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_cpx-assoc-summary_2014to2023-draft.csv"
#ics209_assoc = pd.read_csv(ics209_assoc_fp)

# --- Load ICS-209 incidents ---
ics209_incidents_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\ics209plus-wf_incidents_1999to2023-draft.csv"
#ics209_incidents = pd.read_csv(ics209_incidents_fp)


# Helper
def normalize_id(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip()
    s = s.str.replace(r'\.0+$', '', regex=True)    # drop trailing .0
    s = s.str.replace(r'\D', '', regex=True)       # keep only digits
    return s

# Load
ics209_assoc = pd.read_csv(ics209_assoc_fp, low_memory=False)
ics209_incidents = pd.read_csv(ics209_incidents_fp, low_memory=False)

# Normalize keys
df_SUM_Failures["FOD_ID_norm"] = normalize_id(df_SUM_Failures["FOD_ID"])
ics209_assoc["FOD_ID_norm"]    = normalize_id(ics209_assoc["FOD_ID"])
ics209_assoc["CPLX_INCIDENT_ID_norm"] = normalize_id(ics209_assoc["CPLX_INCIDENT_ID"])
ics209_incidents["INCIDENT_ID_norm"]  = normalize_id(ics209_incidents["INCIDENT_ID"])
if "FOD_ID" in ics209_incidents.columns:
    ics209_incidents["FOD_ID_norm"] = normalize_id(ics209_incidents["FOD_ID"])
else:
    ics209_incidents["FOD_ID_norm"] = pd.Series(pd.NA, index=ics209_incidents.index)

# Merge failures with assoc on FOD_ID_norm
merged = df_SUM_Failures.merge(
    ics209_assoc,
    how="left",
    on="FOD_ID_norm",
    suffixes=("", "_assoc"),
    indicator=False
)

# First try: join incidents on complex id
m1 = merged.merge(
    ics209_incidents,
    how="left",
    left_on="CPLX_INCIDENT_ID_norm",
    right_on="INCIDENT_ID_norm",
    suffixes=("", "_incident")
)

# Rows that did NOT match on complex id
no_match_mask = m1["INCIDENT_ID_norm"].isna()

# Fallback: those rows, join on FOD_ID_norm instead
base_cols = merged.columns  # columns before incident join
fallback_base = merged.loc[no_match_mask, base_cols].copy()
m2 = fallback_base.merge(
    ics209_incidents,
    how="left",
    left_on="FOD_ID_norm",
    right_on="FOD_ID_norm",
    suffixes=("", "_incident")
)

# Combine: complex-id matches + fallback FOD_ID matches
merged = pd.concat([m1.loc[~no_match_mask], m2], ignore_index=True)


In [ ]:
# 0) Sanity: how many complexes repeat?
dup_counts = merged["CPLX_INCIDENT_ID_norm"].value_counts(dropna=True)
print("Complex IDs with >1 rows:", (dup_counts > 1).sum())

# 1) Ensure FIRE_SIZE is numeric
merged["FIRE_SIZE_num"] = pd.to_numeric(merged["FIRE_SIZE"], errors="coerce")

# 2) Rows WITH a complex: keep the row with the max FIRE_SIZE per complex
#    (idxmax is robust and fast)
idx_max = (
    merged.loc[merged["CPLX_INCIDENT_ID_norm"].notna()]
          .groupby("CPLX_INCIDENT_ID_norm")["FIRE_SIZE_num"]
          .idxmax()
)

with_complex = merged.loc[idx_max]

# 3) Rows WITHOUT a complex: keep all
no_complex = merged.loc[merged["CPLX_INCIDENT_ID_norm"].isna()]

# 4) Combine back
merged_final = pd.concat([with_complex, no_complex], ignore_index=True)

# Optional: report reduction
print(f"Before: {len(merged):,}  After: {len(merged_final):,}")


In [ ]:
len(merged_final)

In [ ]:
list(merged_final)

In [ ]:
len(df_SUM_Failures)

In [ ]:
import os

out_dir = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Outputs"
os.makedirs(out_dir, exist_ok=True)
out_fp = os.path.join(out_dir, "merged_failures_ics209.csv")

merged_final.to_csv(out_fp, index=False)
print(f"Wrote CSV to: {out_fp}")
print(f"Rows: {len(merged_final):,} | Columns: {merged_final.shape[1]}")


In [ ]:
STR_DAMAGED_TOTAL, TOTAL_PERSONNEL_SUM, PROJECTED_FINAL_IM_COST

In [ ]:
# import matplotlib.pyplot as plt

# # Make sure the column is numeric
# merged_final["PROJECTED_FINAL_IM_COST"] = pd.to_numeric(
#     merged_final["PROJECTED_FINAL_IM_COST"], errors="coerce"
# )

# # Group by FIRE_YEAR and sum
# cost_by_year = (
#     merged_final.groupby("FIRE_YEAR")["PROJECTED_FINAL_IM_COST"]
#     .sum(min_count=1)   # keep NaN if all values are missing
#     .reset_index()
#     .sort_values("FIRE_YEAR")
# )

# # Plot
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(cost_by_year["FIRE_YEAR"], cost_by_year["PROJECTED_FINAL_IM_COST"],
#         marker="o", color="firebrick", linewidth=2)

# ax.set_xlabel("Year", fontsize=12)
# ax.set_ylabel("Projected Final IM Cost (sum)", fontsize=12)
# ax.set_title("Projected Final IM Cost by Year", fontsize=14)
# ax.grid(axis="y", linestyle="--", alpha=0.5)

# plt.tight_layout()
# plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Make sure the column is numeric
merged_final["PROJECTED_FINAL_IM_COST"] = pd.to_numeric(
    merged_final["PROJECTED_FINAL_IM_COST"], errors="coerce"
)

# Group by FIRE_YEAR and sum (convert to millions)
cost_by_year = (
    merged_final.groupby("FIRE_YEAR")["PROJECTED_FINAL_IM_COST"]
    .sum(min_count=1)   # keep NaN if all values are missing
    .div(1e9)           # convert to millions
    .reset_index()
    .sort_values("FIRE_YEAR")
)

# Plot
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(cost_by_year["FIRE_YEAR"], cost_by_year["PROJECTED_FINAL_IM_COST"],
        marker="o", color="firebrick", linewidth=2)

ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel("Projected Final IM Cost (Billion $)", fontsize=12)
#ax.set_title("Projected Final IM Cost by Year", fontsize=14)
ax.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Make sure the column is numeric
merged_final["STR_DAMAGED_TOTAL"] = pd.to_numeric(
    merged_final["STR_DAMAGED_TOTAL"], errors="coerce"
)

# Group by FIRE_YEAR and sum (convert to thousands if needed, here kept as raw counts)
damaged_by_year = (
    merged_final.groupby("FIRE_YEAR")["STR_DAMAGED_TOTAL"]
    .sum(min_count=1)   
    .reset_index()
    .sort_values("FIRE_YEAR")
)

# Plot
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(damaged_by_year["FIRE_YEAR"], damaged_by_year["STR_DAMAGED_TOTAL"],
        marker="o", color="firebrick", linewidth=2)

ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel("Structures Damaged (count)", fontsize=12)
#ax.set_title("Structures Damaged by Year", fontsize=14)
ax.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

cols = [
    "STR_DAMAGED_COMM_TOTAL",
    "STR_DAMAGED_RES_TOTAL",
    "STR_DAMAGED_TOTAL",
    "STR_DESTROYED_COMM_TOTAL",
    "STR_DESTROYED_RES_TOTAL",
    "STR_DESTROYED_TOTAL",
    "STR_THREATENED_COMM_MAX",
    "STR_THREATENED_MAX",
    "STR_THREATENED_RES_MAX"
]

for col in cols:
    # Make sure numeric
    merged_final[col] = pd.to_numeric(merged_final[col], errors="coerce")

    # Aggregate by year
    yearly = (
        merged_final.groupby("FIRE_YEAR")[col]
        .sum(min_count=1)
        .reset_index()
        .sort_values("FIRE_YEAR")
    )

    # Plot
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(yearly["FIRE_YEAR"], yearly[col],
            marker="o", color="firebrick", linewidth=2)

    ax.set_xlabel("Year", fontsize=12)
    ax.set_ylabel(f"{col} (count)", fontsize=12)
    ax.set_title(f"{col} by Year", fontsize=14)
    ax.grid(axis="y", linestyle="--", alpha=0.5)

    plt.tight_layout()
    plt.show()


In [ ]:
# import matplotlib.pyplot as plt

# # Make sure the column is numeric
# merged_final["TOTAL_PERSONNEL_SUM"] = pd.to_numeric(
#     merged_final["TOTAL_PERSONNEL_SUM"], errors="coerce"
# )

# # Group by FIRE_YEAR and sum
# personnel_by_year = (
#     merged_final.groupby("FIRE_YEAR")["TOTAL_PERSONNEL_SUM"]
#     .sum(min_count=1)   
#     .reset_index()
#     .sort_values("FIRE_YEAR")
# )

# # Plot
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(personnel_by_year["FIRE_YEAR"], personnel_by_year["TOTAL_PERSONNEL_SUM"],
#         marker="o", color="firebrick", linewidth=2)

# ax.set_xlabel("Year", fontsize=12)
# ax.set_ylabel("Total Personnel (sum)", fontsize=12)
# #ax.set_title("Total Personnel by Year", fontsize=14)
# ax.grid(axis="y", linestyle="--", alpha=0.5)

# plt.tight_layout()
# plt.show()


import matplotlib.pyplot as plt

# Make sure the column is numeric
merged_final["TOTAL_PERSONNEL_SUM"] = pd.to_numeric(
    merged_final["TOTAL_PERSONNEL_SUM"], errors="coerce"
)

# Group by FIRE_YEAR and sum (convert to hundred-thousands)
personnel_by_year = (
    merged_final.groupby("FIRE_YEAR")["TOTAL_PERSONNEL_SUM"]
    .sum(min_count=1)
    .div(1e6)   # convert to units of 100,000
    .reset_index()
    .sort_values("FIRE_YEAR")
)

# Plot
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(personnel_by_year["FIRE_YEAR"], personnel_by_year["TOTAL_PERSONNEL_SUM"],
        marker="o", color="firebrick", linewidth=2)

ax.set_xlabel("Year", fontsize=12)
ax.set_ylabel("Total Personnel Days (in millions)", fontsize=12)
#ax.set_title("Total Personnel by Year", fontsize=14)
ax.grid(axis="y", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
# # I put this above
# import geopandas as gpd
# import pandas as pd
# import numpy as np

# # --- Start from merged_final and ensure no sjoin-reserved columns are present ---
# df = merged_final.copy()
# for col in ["index_right", "index_left"]:
#     if col in df.columns:
#         df = df.drop(columns=col)

# # --- Keep only rows with valid coords ---
# mask_xy = df["LATITUDE"].notna() & df["LONGITUDE"].notna()
# df_xy = df.loc[mask_xy].copy()

# # --- Build point GeoDataFrame ---
# gdf_points = gpd.GeoDataFrame(
#     df_xy,
#     geometry=gpd.points_from_xy(df_xy["LONGITUDE"], df_xy["LATITUDE"]),
#     crs="EPSG:4326"
# )

# # --- Load GACC polygons ---
# gacc_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\National_GACC_Final_20250113.shp"
# gdf_gacc = gpd.read_file(gacc_fp)[["GACCAbbrev", "geometry"]].reset_index(drop=True)

# # --- Reproject if needed ---
# if gdf_gacc.crs != gdf_points.crs:
#     gdf_gacc = gdf_gacc.to_crs(gdf_points.crs)

# # --- Spatial join (avoid 'index_right' collision via suffixes) ---
# gdf_joined = gpd.sjoin(
#     gdf_points.reset_index(),               # keep original index to map back
#     gdf_gacc,
#     how="left",
#     predicate="within",
#     lsuffix="PTS",
#     rsuffix="GACC"
# )

# # --- Extract GACCAbbrev as GACCTRUTH and map back to full dataframe ---
# gacc_map = gdf_joined.set_index("index")["GACCAbbrev"]
# df["GACCTRUTH"] = np.nan
# df.loc[gacc_map.index, "GACCTRUTH"] = gacc_map.values

# # --- Put result back into merged_final ---
# merged_final = df

# # Optional quick check:
# # print(merged_final["GACCTRUTH"].value_counts(dropna=False))


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

# --- Start from merged_final and ensure no sjoin-reserved columns are present ---
df = merged_final.copy()
for col in ["index_right", "index_left"]:
    if col in df.columns:
        df = df.drop(columns=col)

# --- Keep only rows with valid coords ---
mask_xy = df["LATITUDE"].notna() & df["LONGITUDE"].notna()
df_xy = df.loc[mask_xy].copy()

# --- Build point GeoDataFrame ---
gdf_points = gpd.GeoDataFrame(
    df_xy,
    geometry=gpd.points_from_xy(df_xy["LONGITUDE"], df_xy["LATITUDE"]),
    crs="EPSG:4326"
)

# --- Load GACC polygons and rename the abbrev column to a unique name ---
gacc_fp = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Data\National_GACC_Final_20250113.shp"
gdf_gacc_raw = gpd.read_file(gacc_fp)

# sanity check: ensure the field exists (it does on your file, but this guards future runs)
if "GACCAbbrev" not in gdf_gacc_raw.columns:
    raise KeyError(f"'GACCAbbrev' not found in {gacc_fp}. Available columns: {list(gdf_gacc_raw.columns)}")

gdf_gacc = gdf_gacc_raw[["GACCAbbrev", "geometry"]].reset_index(drop=True).rename(
    columns={"GACCAbbrev": "GACCTRUTH"}
)

# --- Reproject if needed ---
if gdf_gacc.crs != gdf_points.crs:
    gdf_gacc = gdf_gacc.to_crs(gdf_points.crs)

# --- Spatial join (no suffixes needed now that names are unique) ---
gdf_joined = gpd.sjoin(
    gdf_points.reset_index(),   # keep original index to map back
    gdf_gacc,
    how="left",
    predicate="within",
)

# figure out the left index column name robustly (usually 'index')
left_idx_col = "index" if "index" in gdf_joined.columns else next(
    c for c in gdf_joined.columns if c.startswith("index")
)

# --- Extract GACCTRUTH and map back to full dataframe ---
gacc_map = gdf_joined.set_index(left_idx_col)["GACCTRUTH"]
df["GACCTRUTH"] = np.nan
df.loc[gacc_map.index, "GACCTRUTH"] = gacc_map.values

# --- Put result back into merged_final ---
merged_final = df

# Optional quick check:
# print(merged_final["GACCTRUTH"].value_counts(dropna=False))


In [ ]:
# Count all unique values (including NaN)
print("Value counts for GACCTRUTH (including NaN):")
print(merged_final["GACCTRUTH"].value_counts(dropna=False))

# If you just want the unique list and how many
unique_vals = merged_final["GACCTRUTH"].unique()
print("\nUnique values:", unique_vals)
print("Number of unique values:", merged_final["GACCTRUTH"].nunique(dropna=False))


In [ ]:
import pandas as pd
import numpy as np

# Start from your merged_final
df_SEASONS = merged_final.copy()

# --- Robust DISCOVERY_DATE parsing (handles strings and Excel-serial numbers) ---
disc = pd.to_datetime(df_SEASONS["DISCOVERY_DATE"], errors="coerce")
if disc.isna().mean() > 0.9 and pd.api.types.is_numeric_dtype(df_SEASONS["DISCOVERY_DATE"]):
    disc = pd.to_datetime(df_SEASONS["DISCOVERY_DATE"], unit="D", origin="1899-12-30", errors="coerce")
df_SEASONS["_DISC_DT"] = disc

# --- Map EXACT GACCAbbrev values you have to canonical keys we’ll use in the window table ---
# Unique values you reported: ONCC, GBCC, NWCC, SWCC, NRCC, AICC, SACC, RMCC, OSCC, EACC, NaN
def map_gacc_exact(x):
    if pd.isna(x):
        return "UNK"
    t = str(x).strip().upper()
    mapping = {
        "AICC": "AICC",   # Alaska
        "NWCC": "NWCC",   # Northwest
        "NRCC": "NRCC",   # Northern Rockies
        "RMCC": "RMCC",   # Rocky Mountain
        "GBCC": "GBCC",   # Great Basin
        "SWCC": "SWCC",   # Southwest
        "ONCC": "ONCC",   # North Ops (CA-N)
        "OSCC": "OSCC",   # South Ops (CA-S)
        "EACC": "EACC",   # Eastern (Great Lakes & Northeast)
        "SACC": "SACC",   # Southern (Southeast)
    }
    return mapping.get(t, "UNK")

df_SEASONS["_GACC_KEY"] = df_SEASONS["GACCTRUTH"].map(map_gacc_exact)

# --- Season windows keyed by those exact abbreviations ---
# (month, day) inclusive windows derived from your map
SEASON_WINDOWS = {
    "AICC": [(5, 1, 8, 31)],                       # Alaska: May–Aug
    "NWCC": [(6, 1, 10, 31)],                      # Northwest: Jun–Oct
    "NRCC": [(6, 1, 9, 30)],                       # Northern Rockies: Jun–Sep
    "RMCC": [(6, 1, 9, 30)],                       # Rocky Mountain: Jun–Sep
    "GBCC": [(6, 1, 9, 30)],                       # Great Basin: Jun–Sep
    "SWCC": [(5, 1, 7, 15)],                       # Southwest: May–mid July
    "ONCC": [(5, 1, 10, 31)],                      # North CA (North Ops): May–Oct
    "OSCC": [(5, 1, 10, 31)],                      # South CA (South Ops): May–Oct
    "EACC": [(3, 1, 5, 31), (10, 1, 11, 30)],      # Great Lakes & Northeast: Mar–May, Oct–Nov
    "SACC": [(2, 1, 5, 31), (10, 1, 11, 30)],      # Southeast: Feb–May, Oct–Nov
    "UNK":  []                                      # Unknown/NaN -> treat as out of season
}

# --- Helper: safe window check ---
def in_windows(dt, windows):
    if pd.isna(dt) or not windows:
        return False
    m, d = dt.month, dt.day
    for sm, sd, em, ed in windows:
        if (sm, sd) <= (em, ed):  # same-year window
            if (m, d) >= (sm, sd) and (m, d) <= (em, ed):
                return True
        else:  # wrap-around window (not used here)
            if (m, d) >= (sm, sd) or (m, d) <= (em, ed):
                return True
    return False

# Map each row's GACC to its windows; default to [] for unknowns/NaN
gacc_windows = df_SEASONS["_GACC_KEY"].map(SEASON_WINDOWS)

# Compute SEASON flag (1 in-season, 0 out-of-season)
df_SEASONS["SEASON"] = np.fromiter(
    (in_windows(dt, wins) for dt, wins in zip(df_SEASONS["_DISC_DT"], gacc_windows)),
    dtype=bool
).astype("int8")

# Cleanup helpers
df_SEASONS.drop(columns=["_DISC_DT", "_GACC_KEY"], inplace=True)


In [ ]:
df_SEASONS

import os

out_dir = r"C:\Users\magst\OneDrive\Pictures\Desktop\IA_CONTAINMENT\Outputs"
os.makedirs(out_dir, exist_ok=True)
out_fp = os.path.join(out_dir, "merged_failures_ics209_SEASONS.csv")

df_SEASONS.to_csv(out_fp, index=False)
print(f"Wrote CSV to: {out_fp}")
print(f"Rows: {len(merged_final):,} | Columns: {merged_final.shape[1]}")


In [ ]:
import matplotlib.pyplot as plt

# --- Aggregate counts by FIRE_YEAR and SEASON ---
season_counts = (
    df_SEASONS.groupby(["FIRE_YEAR", "SEASON"])
    .size()
    .reset_index(name="count")
    .pivot(index="FIRE_YEAR", columns="SEASON", values="count")
    .fillna(0)
)

# Ensure both 0 and 1 columns exist
for col in [0, 1]:
    if col not in season_counts.columns:
        season_counts[col] = 0

# --- Plot grouped (side-by-side) bar chart ---
fig, ax = plt.subplots(figsize=(12, 6))
season_counts[[0, 1]].plot(
    kind="bar",
    stacked=False,
    ax=ax,
    width=0.8,
    color={0: "gray", 1: "firebrick"}
)

ax.set_xlabel("Fire Year", fontsize=12)
ax.set_ylabel("Number of Fires", fontsize=12)
ax.set_title("Fires In-Season vs Out-of-Season by Year", fontsize=14, weight="bold")
ax.legend(["Out of Season (0)", "In Season (1)"], title="Season", loc="upper right")

plt.tight_layout()
plt.show()
